# Fit energies

In [ ]:
from rascal.representations import SphericalInvariants



In [ ]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
traj_355 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")

traj_295_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
traj_355_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
traj_295_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
traj_355_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")



In [ ]:
traj_train = traj_295[0:5000:10] + traj_355[0:5000:10] + traj_295_2000K[0:5000:10] + traj_355_2000K[0:5000:10] + traj_295_2000K_forced[0:5000:10] + traj_355_2000K_forced[0:5000:10]

In [ ]:
energies_train = [s.get_potential_energy() for s in traj_train]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(energies)
plt.show()


In [ ]:
import math
from tqdm import tqdm

def prepare_batches(
        # self, 
        traj, 
        forces,
        energies, 
        frames_per_batch=10):
        
        n_frames = len(traj)
        n_batches = math.ceil(n_frames/frames_per_batch)

        print(f"Total number of frames is {n_frames}")
        print(f"Total number of batches is {n_batches}")

        batches = []
        for i in range(n_batches):
            batch = {}
            batch["traj"] = traj[i*frames_per_batch:(i+1)*frames_per_batch]
        #     batch["forces"] = forces[i*frames_per_batch:(i+1)*frames_per_batch]
            batch["energies"] = energies[i*frames_per_batch:(i+1)*frames_per_batch]
            batches.append(batch)

        return batches


soap_params = {
# 'species': ["H", "C", "O", "N", "Si"],
# 'periodic': True,
'interaction_cutoff': 3.0, #5
'gaussian_sigma_constant': 0.3,
'max_radial': 4, #5
'max_angular': 4,#5
'cutoff_smooth_width': 0.1,
# 'average': "off",
# 'crossover': True,
# 'dtype': "float64",
# 'n_jobs': 10,
# 'sparse': False,
# 'positions': [7, 11, 15] # ignored
}


# species= soap_params['species']
# periodic= soap_params['periodic']
interaction_cutoff = soap_params['interaction_cutoff']
gaussian_sigma_constant= soap_params['gaussian_sigma_constant']
max_radial= soap_params['max_radial']
max_angular= soap_params['max_angular']
cutoff_smooth_width = soap_params['cutoff_smooth_width']
# average= soap_params['average']
# crossover= soap_params['crossover']
# dtype= soap_params['dtype']
# sparse= soap_params['sparse']
# positions = soap_params['positions']

hypers = dict(soap_type="PowerSpectrum",
        interaction_cutoff=interaction_cutoff,
        max_radial=max_radial,
        max_angular=max_angular,
        gaussian_sigma_constant=gaussian_sigma_constant,
        gaussian_sigma_type="Constant",
        cutoff_function_type="RadialScaling",
        cutoff_smooth_width=cutoff_smooth_width, # 0.1 is way better than 0.5
        cutoff_function_parameters=
                dict(
                        rate=1,
                        scale=3.5,
                        exponent=4
                        ),
        radial_basis="GTO",
        normalize=False, # setting False makes model untrainable
        #   optimization=
        #         dict(
        #                 Spline=dict(
        #                    accuracy=1.0e-05
        #                 )
        #             ),
        compute_gradients=True,
        expansion_by_species_method='structure wise'
        )

for f in traj:
        f.wrap(eps=1e-18)

n_atoms = len(traj[0])

frames_batches = prepare_batches(traj, [0]*len(traj), energies, frames_per_batch=1)



print(f"Total length of traj is {len(traj)}")
print(f"Total number of batches {len(frames_batches)}")       
print("Calculating invariants on trajectory with librascal...")

soap = SphericalInvariants(**hypers)

X_np_batched = []
E_np_batched = []

# DX_np_batched = [[] * len(frames_batches) for i in range(n_atomic_groups)]  
# F_np_batched = [[] * len(frames_batches) for i in range(n_atomic_groups)]
# grad_info_sub_batched = [[] * len(frames_batches) for i in range(n_atomic_groups)]

for ind_b, batch in enumerate(tqdm(frames_batches)):
        traj_b = batch['traj']
        # forces_b = batch['forces']
        energies_b = batch['energies']

        managers = soap.transform(traj_b)
        soap_array = managers.get_features(soap)
        X_np_batched.append(soap_array)
        E_np_batched.append(energies_b)

In [ ]:
indices_non_H = []

for ind,s in enumerate(traj[0].get_chemical_symbols()):
        # print(s)
        if s != "H":
                indices_non_H.append(ind)

In [ ]:
import numpy as np

X_np_batched = np.array(X_np_batched)
E_np_batched = np.array(E_np_batched).flatten()

E_np_batched_no_mean = E_np_batched - np.mean(E_np_batched)

In [ ]:
energies_train_normalized = (energies_train - np.min(energies_train))/(np.max(energies_train) - np.min(energies_train))

In [ ]:
energies_train_normalized = energies_train_normalized[100:]

In [ ]:
energies_train_normalized.shape

In [ ]:
train_X = X_np_batched[:,indices_non_H,:]

In [ ]:
train_X.shape

In [ ]:
import torch

X_np_batched_stacked = X_np_batched.reshape(-1, X_np_batched.shape[-1]*X_np_batched.shape[-2])

# train_X = torch.tensor(X_np_batched)
train_X = train_X.sum(axis=1)
train_Y = torch.tensor(E_np_batched_no_mean)



test_X = train_X[0:100]
test_Y = train_Y[0:100]

train_X = train_X[100:]
train_Y = train_Y[100:]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

train_X = torch.tensor(train_X)
train_Y = torch.tensor(train_Y)
test_X = torch.tensor(test_X)
test_Y = torch.tensor(test_Y)


In [ ]:
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

In [ ]:
# 1 flatten the per snapshot collection of invariants
c = train_X.reshape(-1, train_X.size(2))
# test_X.shape
print(c.shape)

# 2. check that the flattened invariants are the same as the original invariants
i = 10
(train_X[i] == c[i*train_X.size(1):(i+1)*train_X.size(1)]).all()

In [ ]:
# 3. example kernel matrix
kern = c@c.T
kern = kern.unsqueeze(0)

print(kern.shape)

In [ ]:
21120/528
# train_X.shape

In [ ]:
40*528

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html
# https://stackoverflow.com/questions/63240702/torch-sum-subsets-of-tensor
import torch, torch.nn as nn

# pool of square window of size=3, stride=2
m = nn.AvgPool2d(528, stride=528)
# pool of non-square window
# m = nn.AvgPool2d((3, 3), stride=(2, 1))
# input = torch.randn(1, 528, 528)
input = kern
output = m(input)
print(output.shape)

In [ ]:
nn.AvgPool1d

In [ ]:
# 4. perform block summation of the kernel matrix



In [ ]:
import torch
import gpytorch
import numpy as np

from gpytorch.kernels import (
    RBFKernel,
    ScaleKernel,
    LinearKernel,
    AdditiveKernel,
    MultitaskKernel,
    PolynomialKernel,
    MaternKernel,
)
from gpytorch.means import ZeroMean, ConstantMean

from gpytorch.models import ExactGP

import torch, torch.nn as nn


# https://stackoverflow.com/questions/76251549/setting-the-task-covariance-matrix-to-the-correlation-matrix-in-gpytorch

class ExactGPModelEnergies(ExactGP):
    def __init__(self, train_X, train_Y, likelihood):
        super().__init__(
            train_X, train_Y, likelihood
        )  # the old-style super(ExactGPModel, self) was causing error!
        self.mean_module = ConstantMean()

        self.soap_dim = train_X.shape[-1]
        # self.covar_module = ScaleKernel(RBFKernel(ard_num_dims=self.soap_dim))#LinearKernel()
        self.covar_module = ScaleKernel(MaternKernel(ard_num_dims=self.soap_dim))#LinearKernel()
        # self.covar_module = ScaleKernel(LinearKernel(ard_num_dims=self.soap_dim))

    def forward(self, X):
        x = X
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)



In [ ]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
# model = ExactGPModelEnergies(train_X, train_Y, likelihood)
model = ExactGPModelEnergiesMulti(train_X, train_Y, likelihood)

In [ ]:
train_X = train_X.cuda()
train_Y = torch.tensor(energies_train_normalized).cuda()
test_X = test_X.cuda()
test_Y = test_Y.cuda()
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModelEnergies(train_X, train_Y, likelihood)
# model = ExactGPModelEnergiesMulti(train_X, train_Y, likelihood)
model = model.cuda()
likelihood = likelihood.cuda()

In [ ]:
train_Y.shape

In [ ]:
# this is for running the notebook in our testing framework
import os



training_iter = 300


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

print("Training...")
# initial_loss = -mll(model(train_X), train_Y).item()
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # print("grad zeroed")
    # Output from model
    output = model(train_X)
    # print("Output calculated")
    # print(output)
    # Calc loss and backprop gradients
    loss = -mll(output, train_Y)
    # print("Loss calculated, ")

    loss.backward()
    # print(loss)
    # print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
    #     i + 1, training_iter, loss.item(),
    #     model.covar_module.base_kernel.lengthscale.item(),
    #     model.likelihood.noise.item()
    # ))
    print(f"step: {i},loss: {loss}")
    optimizer.step()

# print("Initial loss: ", initial_loss)
print("Final loss: ", loss)


In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

test_x = test_X

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
# with torch.no_grad(), gpytorch.settings.fast_pred_var():
#     test_x = torch.linspace(0, 1, 51)
observed_pred = likelihood(model(test_x)).mean.detach().cpu().numpy()



import matplotlib.pyplot as plt

plt.plot(observed_pred, label="Predicted energy")
plt.plot(test_Y.detach().cpu().numpy(), label="True energy")
plt.xlabel('step')
plt.ylabel('Predicted Energy')
# plt.xlim(0, 100)
plt.legend()
plt.show()

In [ ]:
plt.plot(test_Y)
plt.ylabel('Energy')
plt.xlabel('step')
plt.xlim(85, 100)
plt.show()

In [ ]:
traj_test = io.read("/data1/simulations/datasets/rotors/for jacs paper/295_0.075_same_+/md_trajectory.traj", index=":", format="traj")

In [ ]:
energies_test = [s.get_potential_energy() for s in traj_test]

In [ ]:
energies_test_normalized = (energies_test - np.min(energies_train))/(np.max(energies_train) - np.min(energies_train))

In [ ]:
traj = traj_test

In [ ]:
import math
from tqdm import tqdm

def prepare_batches(
        # self, 
        traj, 
        forces,
        energies, 
        frames_per_batch=10):
        
        n_frames = len(traj)
        n_batches = math.ceil(n_frames/frames_per_batch)

        print(f"Total number of frames is {n_frames}")
        print(f"Total number of batches is {n_batches}")

        batches = []
        for i in range(n_batches):
            batch = {}
            batch["traj"] = traj[i*frames_per_batch:(i+1)*frames_per_batch]
        #     batch["forces"] = forces[i*frames_per_batch:(i+1)*frames_per_batch]
            batch["energies"] = energies[i*frames_per_batch:(i+1)*frames_per_batch]
            batches.append(batch)

        return batches


soap_params = {
# 'species': ["H", "C", "O", "N", "Si"],
# 'periodic': True,
'interaction_cutoff': 3.0, #5
'gaussian_sigma_constant': 0.3,
'max_radial': 4, #5
'max_angular': 4,#5
'cutoff_smooth_width': 0.1,
# 'average': "off",
# 'crossover': True,
# 'dtype': "float64",
# 'n_jobs': 10,
# 'sparse': False,
# 'positions': [7, 11, 15] # ignored
}


# species= soap_params['species']
# periodic= soap_params['periodic']
interaction_cutoff = soap_params['interaction_cutoff']
gaussian_sigma_constant= soap_params['gaussian_sigma_constant']
max_radial= soap_params['max_radial']
max_angular= soap_params['max_angular']
cutoff_smooth_width = soap_params['cutoff_smooth_width']
# average= soap_params['average']
# crossover= soap_params['crossover']
# dtype= soap_params['dtype']
# sparse= soap_params['sparse']
# positions = soap_params['positions']

hypers = dict(soap_type="PowerSpectrum",
        interaction_cutoff=interaction_cutoff,
        max_radial=max_radial,
        max_angular=max_angular,
        gaussian_sigma_constant=gaussian_sigma_constant,
        gaussian_sigma_type="Constant",
        cutoff_function_type="RadialScaling",
        cutoff_smooth_width=cutoff_smooth_width, # 0.1 is way better than 0.5
        cutoff_function_parameters=
                dict(
                        rate=1,
                        scale=3.5,
                        exponent=4
                        ),
        radial_basis="GTO",
        normalize=False, # setting False makes model untrainable
        #   optimization=
        #         dict(
        #                 Spline=dict(
        #                    accuracy=1.0e-05
        #                 )
        #             ),
        compute_gradients=True,
        expansion_by_species_method='structure wise'
        )

for f in traj:
        f.wrap(eps=1e-18)

n_atoms = len(traj[0])

frames_batches = prepare_batches(traj, [0]*len(traj), energies, frames_per_batch=1)



print(f"Total length of traj is {len(traj)}")
print(f"Total number of batches {len(frames_batches)}")       
print("Calculating invariants on trajectory with librascal...")

soap = SphericalInvariants(**hypers)

X_np_batched = []
E_np_batched = []

# DX_np_batched = [[] * len(frames_batches) for i in range(n_atomic_groups)]  
# F_np_batched = [[] * len(frames_batches) for i in range(n_atomic_groups)]
# grad_info_sub_batched = [[] * len(frames_batches) for i in range(n_atomic_groups)]

for ind_b, batch in enumerate(tqdm(frames_batches)):
        traj_b = batch['traj']
        # forces_b = batch['forces']
        energies_b = batch['energies']

        managers = soap.transform(traj_b)
        soap_array = managers.get_features(soap)
        X_np_batched.append(soap_array)
        E_np_batched.append(energies_b)

In [ ]:
indices_non_H = []

for ind,s in enumerate(traj[0].get_chemical_symbols()):
        # print(s)
        if s != "H":
                indices_non_H.append(ind)

In [ ]:
import numpy as np

X_np_batched = np.array(X_np_batched)
E_np_batched = np.array(E_np_batched).flatten()

E_np_batched_no_mean = E_np_batched - np.mean(energies_train)

In [ ]:
test_X = X_np_batched[:,indices_non_H,:]
test_X = test_X.sum(axis=1)
test_X = torch.tensor(test_X).cuda()
test_Y = torch.tensor(energies_test_normalized).cuda()


In [ ]:
from matplotlib import pyplot as plt

plt.plot(energies_train_normalized)
# plt.plot(energies_test_normalized)
plt.plot(test_Y.detach().cpu().numpy())
plt.show()